https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [64]:
from birdcall_distribution import geo
import json
from shapely.geometry import mapping

ca_shape = geo.get_shape_us_state("California")
grid = geo.generate_grid(ca_shape, (-124, -114, 32, 42), (0.25, 0.25))

In [65]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection("MODIS/006/MCD12Q1")

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection("MODIS/006/MOD11A1")

# Import the USGS ground elevation image.
elv = ee.Image("USGS/SRTMGL1_003")

# Initial date of interest (inclusive).
# Final date of interest (exclusive).
i_date = "2017-01-01"
f_date = "2020-01-01"

# temperature day and night, with quality control bands
lst = lst.select("LST_Day_1km", "LST_Night_1km", "QC_Day", "QC_Night").filterDate(
    i_date, f_date
)

## get stats for a single polygon

In [83]:
polygons = [ee.Geometry.Polygon(mapping(p)["coordinates"]) for p in grid.values()]
polygon = polygons[0]

In [90]:
# scale in meters
scale = 1000

# elevation data
elv.sample(polygon, scale).aggregate_stats("elevation").getInfo()

{'max': 122,
 'mean': 1.6581632653061225,
 'min': 0,
 'sample_sd': 11.88140565983313,
 'sample_var': 141.16780045351473,
 'sum': 1300,
 'sum_sq': 112690,
 'total_count': 784,
 'total_sd': 11.873825814940545,
 'total_var': 140.98773948354852,
 'valid_count': 784,
 'weight_sum': 784,
 'weighted_sum': 1300}

In [99]:
# LST collection (land surface temperature)
for property in ["LST_Day_1km", "LST_Night_1km"]:
    temp = lst.mean().sample(polygon, scale).aggregate_mean(property).getInfo()
    print(property, temp)

LST_Day_1km 14386.504953502432
LST_Night_1km 14071.905977195267


In [69]:
lc_point = lc.first().sample(polygon, scale).first().getInfo()
lc_point

{'type': 'Feature',
 'geometry': None,
 'id': '0',
 'properties': {'LC_Prop1': 43,
  'LC_Prop1_Assessment': 93,
  'LC_Prop2': 40,
  'LC_Prop2_Assessment': 93,
  'LC_Prop3': 40,
  'LC_Prop3_Assessment': 93,
  'LC_Type1': 7,
  'LC_Type2': 7,
  'LC_Type3': 2,
  'LC_Type4': 1,
  'LC_Type5': 5,
  'LW': 2,
  'QC': 0}}

In [100]:
lc.select("LC_Type1").first().sample(polygon, scale).aggregate_histogram(
    "LC_Type1"
).getInfo()

{'1': 7, '11': 2, '17': 591, '2': 3, '5': 1, '7': 1}